In [1]:
from db_communication import db_communication
import numpy as np
import pandas as pd
from datetime import datetime

### Init tables/ dataframes

In [2]:
config = {
    'user': 'root',
    'password': 'FDS-apm1',
    'host': 'min-ifm-xdm.ad.fh-bielefeld.de',
    'port': '3306',
    'database': 'NeueDB' 
}
my_db = db_communication(config)

Successfully connected.


In [3]:
cluster_df = my_db.get_table('Merkmalcluster')

In [4]:
parts_df = my_db.get_table('Einzelteile')

In [5]:
features_df = my_db.get_table('Merkmale')

In [6]:
config_df = my_db.get_table('Konfiguration')
config_df.index = pd.MultiIndex.from_arrays([config_df['AuftragNr'],config_df['KonfigNr']])
config_df = config_df['MerkmalNr'].to_frame()
config_df.head(5)

MerkmalNr
AuftragNr KonfigNr           
1         100          100007
          101          200001
          102          300001
          103          400006
          104          500002

In [7]:
parts_df['Bezeichnung']

0            Surly Long Haul Trucker Rahmenkit
1                        Surly Troll Rahmenkit
2                    Salsa Cutthroat Rahmenkit
3                           VOTEC VRC Framekit
4                          Cervelo C3 Frameset
                        ...                   
117                         Xpedo Detox Pedale
118                NOW8 M36 Flat Pedals 6 Pins
119                         Moto Reflex Pedale
120             Time ATAC XC6 X-Country Pedale
121    Shimano Dura Ace PD-R9100 Pedale carbon
Name: Bezeichnung, Length: 122, dtype: object

In [8]:
features_df.transpose().loc['Bezeichnung']

0              Herren Citybike
1          Herren Trekkingbike
2       Herren Trekking Carbon
3     Unisex Rennrad Aluminium
4        Unisex Rennrad Carbon
                ...           
69             Trekking Pedale
70                Sport Pedale
71              Komfort Pedale
72               Rennradpedale
73           Rennradpedale Pro
Name: Bezeichnung, Length: 74, dtype: object

In [9]:
cluster_df.head(5)

,ClusterNr,Merkmalsbezeichnung
0,1,Rahmen
1,2,Federung
2,3,Lackierung
3,4,Lenker
4,5,Felgen


### Direct demand matrix

In [10]:
# generate Multiindex for the columns
col_main = cluster_df['Merkmalsbezeichnung'][features_df['Merkmalcluster'].values-1]
col_main.name = 'Cluster'
col_sub = features_df['Bezeichnung']
col_sub.name = 'Merkmal'
col = pd.MultiIndex.from_arrays([col_main,col_sub])

In [ ]:
# part name as index 
index_names = parts_df['Bezeichnung']
index_names.name = 'Einzelteil'

In [12]:
# fill data with NaNs
data = np.ones((len(index_names),len(col))) * np.nan

df = pd.DataFrame(data=data,columns=col,index=index_names)
df

Cluster                                          Rahmen                      \
Merkmal                                 Herren Citybike Herren Trekkingbike   
Einzelteil                                                                    
Surly Long Haul Trucker Rahmenkit                   NaN                 NaN   
Surly Troll Rahmenkit                               NaN                 NaN   
Salsa Cutthroat Rahmenkit                           NaN                 NaN   
VOTEC VRC Framekit                                  NaN                 NaN   
Cervelo C3 Frameset                                 NaN                 NaN   
...                                                 ...                 ...   
Xpedo Detox Pedale                                  NaN                 NaN   
NOW8 M36 Flat Pedals 6 Pins                         NaN                 NaN   
Moto Reflex Pedale                                  NaN                 NaN   
Time ATAC XC6 X-Country Pedale                      NaN                 NaN   
Shimano Dura Ace PD-R9100 Pedale carbon             NaN                 NaN   

Cluster                                                         \
Merkmal                                 Herren Trekking Carbon   
Einzelteil                                                       
Surly Long Haul Trucker Rahmenkit                          NaN   
Surly Troll Rahmenkit                                      NaN   
Salsa Cutthroat Rahmenkit                                  NaN   
VOTEC VRC Framekit                                         NaN   
Cervelo C3 Frameset                                        NaN   
...                                                        ...   
Xpedo Detox Pedale                                         NaN   
NOW8 M36 Flat Pedals 6 Pins                                NaN   
Moto Reflex Pedale                                         NaN   
Time ATAC XC6 X-Country Pedale                             NaN   
Shimano Dura Ace PD-R9100 Pedale carbon                    NaN   

Cluster                                                           \
Merkmal                                 Unisex Rennrad Aluminium   
Einzelteil                                                         
Surly Long Haul Trucker Rahmenkit                            NaN   
Surly Troll Rahmenkit                                        NaN   
Salsa Cutthroat Rahmenkit                                    NaN   
VOTEC VRC Framekit                                           NaN   
Cervelo C3 Frameset                                          NaN   
...                                                          ...   
Xpedo Detox Pedale                                           NaN   
NOW8 M36 Flat Pedals 6 Pins                                  NaN   
Moto Reflex Pedale                                           NaN   
Time ATAC XC6 X-Country Pedale                               NaN   
Shimano Dura Ace PD-R9100 Pedale carbon                      NaN   

Cluster                                                                       \
Merkmal                                 Unisex Rennrad Carbon Damen Citybike   
Einzelteil                                                                     
Surly Long Haul Trucker Rahmenkit                         NaN            NaN   
Surly Troll Rahmenkit                                     NaN            NaN   
Salsa Cutthroat Rahmenkit                                 NaN            NaN   
VOTEC VRC Framekit                                        NaN            NaN   
Cervelo C3 Frameset                                       NaN            NaN   
...                                                       ...            ...   
Xpedo Detox Pedale                                        NaN            NaN   
NOW8 M36 Flat Pedals 6 Pins                               NaN            NaN   
Moto Reflex Pedale                                        NaN            NaN   
Time ATAC XC6 X-Country Pedale                         

In [24]:
# save template as Excel with current datetime as version
df.to_excel('DirectDemandMatrix_' + now.strftime('%Y%m%d_%H%M%S') + '.xlsx')

In [25]:
features_df[features_df['Merkmalcluster'] == 4]

,MerkmalNr,Merkmalcluster,Bezeichnung,Preis
23,400001,4,Lenker Standard,12.90
24,400002,4,Lenker Komfort,15.00
25,400003,4,Lenker Retro,18.90
26,400004,4,Rennradlenker,37.99
27,400005,4,Lenker Standard mit Hörnern,48.99
28,400006,4,Lenker Komfort mit Sportklingel,19.49
29,400007,4,Lenker Retro mit Hupe,21.70
30,400008,4,Rennradlenker ohne Klingel,30.00
